In [1]:
import numpy as np
import pandas as pd
import os
import pickle
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import warnings

warnings.filterwarnings('ignore')

# Base directory for the input CSVs
base_dir = "../Data Preprocessing /ProcessedData"  # Change this to your root directory with processed datasets
output_dir = "./Prophet_Models"  # Directory to save trained models
os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

# List of pollutants to include in training
pollutants = ['AQI', 'pm25', 'pm10', 'no2', 'so2', 'co', 'o3']  # Extend as needed

# Helper function: Remove outliers using IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    mask = ~((df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR)))
    return df[mask]

# Iterate through each city
for city in os.listdir(base_dir):
    city_path = os.path.join(base_dir, city)
    
    if os.path.isdir(city_path):
        print(f"\nProcessing city: {city}")
        
        # Create a folder for the city's models
        city_model_dir = os.path.join(output_dir, city)
        os.makedirs(city_model_dir, exist_ok=True)
        
        # Iterate through each CSV file in the city folder
        for csv_file in os.listdir(city_path):
            csv_path = os.path.join(city_path, csv_file)
            
            if csv_file.endswith('.csv'):
                print(f"  Training model for region: {csv_file}")

                # Read the CSV file
                df_city = pd.read_csv(csv_path)

                # Dictionary to store models and metrics
                models = {}
                metrics = {}

                # Train a Prophet model for each pollutant
                for pollutant in pollutants:
                    if pollutant in df_city.columns:
                        print(f"    Training model for pollutant: {pollutant}")
                        
                        # Prepare the data for Prophet
                        df_pollutant = df_city[['date', pollutant]].dropna()
                        df_pollutant.reset_index(drop=True, inplace=True)
                        df_pollutant.rename(columns={'date': 'ds', pollutant: 'y'}, inplace=True)
                        
                        # Ensure 'ds' is in datetime format
                        df_pollutant['ds'] = pd.to_datetime(df_pollutant['ds'])
                        
                        # Remove outliers
                        df_pollutant = remove_outliers(df_pollutant, 'y')

                        # Log transformation (optional for pollutants with high variability)
                        df_pollutant['y'] = np.log1p(df_pollutant['y'])  # Use log1p to handle zeros
                        
                        # Define the Prophet model
                        model = Prophet(
                            seasonality_mode='multiplicative',  # Multiplicative seasonality for pollutants
                            holidays_prior_scale=10,
                            seasonality_prior_scale=15,
                            changepoint_prior_scale=0.05,
                            n_changepoints=75
                        )
                        
                        # Train the model
                        try:
                            model.fit(df_pollutant)
                            models[pollutant] = model
                            print(f"    Model training successful for {pollutant}")
                            
                            # Cross-validation
                            print(f"    Performing cross-validation for {pollutant}")
                            df_cv = cross_validation(
                                model, 
                                initial='900 days',  # Reduced to allow more training data
                                period='180 days', 
                                horizon='365 days'
                            )
                            
                            # Performance metrics
                            df_p = performance_metrics(df_cv)
                            mape = df_p['mape'].mean()
                            accuracy = (1 - mape) * 100
                            metrics[pollutant] = accuracy
                            print(f"    Cross-validation accuracy for {pollutant}: {accuracy:.2f}%")
                        
                        except Exception as e:
                            print(f"    Training or cross-validation failed for {pollutant}: {e}")
                
                # Save the trained models and metrics
                region_model_path = os.path.join(city_model_dir, f"{os.path.splitext(csv_file)[0]}_prophet_model.pkl")
                with open(region_model_path, 'wb') as file:
                    pickle.dump(models, file)
                print(f"    Saved trained models to {region_model_path}")


Processing city: Bangalore
  Training model for region: btm,-bangalore-air-quality.csv
    Training model for pollutant: AQI


20:52:13 - cmdstanpy - INFO - Chain [1] start processing
20:52:16 - cmdstanpy - INFO - Chain [1] done processing


    Model training successful for AQI
    Performing cross-validation for AQI


  0%|          | 0/16 [00:00<?, ?it/s]

20:52:16 - cmdstanpy - INFO - Chain [1] start processing
20:52:17 - cmdstanpy - INFO - Chain [1] done processing
20:52:17 - cmdstanpy - INFO - Chain [1] start processing
20:52:17 - cmdstanpy - INFO - Chain [1] done processing
20:52:17 - cmdstanpy - INFO - Chain [1] start processing
20:52:18 - cmdstanpy - INFO - Chain [1] done processing
20:52:18 - cmdstanpy - INFO - Chain [1] start processing
20:52:18 - cmdstanpy - INFO - Chain [1] done processing
20:52:18 - cmdstanpy - INFO - Chain [1] start processing
20:52:19 - cmdstanpy - INFO - Chain [1] done processing
20:52:19 - cmdstanpy - INFO - Chain [1] start processing
20:52:20 - cmdstanpy - INFO - Chain [1] done processing
20:52:20 - cmdstanpy - INFO - Chain [1] start processing
20:52:21 - cmdstanpy - INFO - Chain [1] done processing
20:52:21 - cmdstanpy - INFO - Chain [1] start processing
20:52:22 - cmdstanpy - INFO - Chain [1] done processing
20:52:22 - cmdstanpy - INFO - Chain [1] start processing
20:52:23 - cmdstanpy - INFO - Chain [1]

...

20:59:46 - cmdstanpy - INFO - Chain [1] done processing


    Model training successful for o3
    Performing cross-validation for o3


  0%|          | 0/10 [00:00<?, ?it/s]

20:59:46 - cmdstanpy - INFO - Chain [1] start processing
20:59:46 - cmdstanpy - INFO - Chain [1] done processing
20:59:46 - cmdstanpy - INFO - Chain [1] start processing
20:59:47 - cmdstanpy - INFO - Chain [1] done processing
20:59:47 - cmdstanpy - INFO - Chain [1] start processing
20:59:47 - cmdstanpy - INFO - Chain [1] done processing
20:59:47 - cmdstanpy - INFO - Chain [1] start processing
20:59:47 - cmdstanpy - INFO - Chain [1] done processing
20:59:47 - cmdstanpy - INFO - Chain [1] start processing
20:59:47 - cmdstanpy - INFO - Chain [1] done processing
20:59:48 - cmdstanpy - INFO - Chain [1] start processing
20:59:48 - cmdstanpy - INFO - Chain [1] done processing
20:59:48 - cmdstanpy - INFO - Chain [1] start processing
20:59:48 - cmdstanpy - INFO - Chain [1] done processing
20:59:48 - cmdstanpy - INFO - Chain [1] start processing
20:59:49 - cmdstanpy - INFO - Chain [1] done processing
20:59:49 - cmdstanpy - INFO - Chain [1] start processing
20:59:49 - cmdstanpy - INFO - Chain [1]

    Cross-validation accuracy for o3: 57.66%
    Saved trained models to ./Prophet_Models/Hyderabad/ida-pashamylaram, hyderabad-air-quality_prophet_model.pkl
